In [6]:
import pandas as pd
import json
import requests 
import time
import random
import tqdm
from urllib.parse import urlparse

In [7]:
# cookies = {
#     '_trackity' : '74634137-8702-8b12-fe42-46a46a2a2573',
#     'TOKENS' : '{%22access_token%22:%22Ae1PtIuUDbozkgpQ6VOW57jKqvchZxF2%22%2C%22expires_in%22:157680000%2C%22expires_at%22:1844582643238%2C%22guest_token%22:%22Ae1PtIuUDbozkgpQ6VOW57jKqvchZxF2%22}',
#     'delivery_zone' : 'Vk4wMzkwMDYwMDE=',
#     '_gid' : 'GA1.2.1038835342.1686902648',
#     'tiki_client_id' : '1000198359.1686902648',
#     '_gcl_au' : '1.1.712561686.1686902651',
#     '_fbp' : 'fb.1.1686902651246.1147493922',
#     '_hjFirstSeen' : '1', 
#     '_hjIncludedInSessionSample_522327' : '0', 
#     '_hjSession_522327':'eyJpZCI6IjRmYmI5NDEzLTc1MzAtNDA3Yy1iYTMyLTYzZTI4Y2MzZTA5NSIsImNyZWF0ZWQiOjE2ODY5MDI2NTEzMTksImluU2FtcGxlIjpmYWxzZX0=',
#     '_hjAbsoluteSessionInProgress' : '0',
#     '__uidac' : 'b0f5fe8e77ade31f60d53549702ce9ac',
#     '__iid' : '749', '__iid' : '749', '__su' : '0',
#     '__su':'0', '__RC' : '5', '__R' : '3',
#     '_hjSessionUser_522327':'eyJpZCI6IjFiMTI5YzM0LWE2NTItNWZmOC05NmEzLTFhY2JlOTMwZTViYyIsImNyZWF0ZWQiOjE2ODY5MDI2NTEzMDIsImV4aXN0aW5nIjp0cnVlfQ==',
#     '__tb':'0', 
#     '__IP' : '712479999',
#     'TIKI_RECOMMENDATION' : '40be1840f8c66757ac3812da38a02d7b',
#     'TKSESSID':'cf67caae16e0362971a2fe1e3367c12e',
#     '_bs':'c4f97c3c-1a3b-0d7a-f2dd-a0a3516f6617',
#     '_gat' : '1',
#     '_ga': 'GA1.2.1000198359.1686902648',
#     '__uif' :'__uid%3A3569195332985378685%7C__ui%3A1%252C5%7C__create%3A1669195332',
#     'cto_bundle':'5mSn7195TEwza3VCTEZZTk9HdWw1cSUyQkM2dGRGV0ZtUlRua1NvRjV5S2phc096S1lIVlJySjlmZkdqUVNScWNMbEdoamc0bE82czlSR0lYS1pKMU1pNk40U2lnQSUyRiUyQlBGbFJ5MXBwUUtwOW1oVUoxQm9hY1olMkZ1ZWplRzhUdThiZnNqeExlMnkyV1NyMnpBNmlJdXAwRFZvb2FsQSUzRCUzRA',
#     'amp_99d374':'Rz66229-5rdmltAOjfyUvJ...1h31jn9b2.1h31k9l0t.2j.5e.81',
#     '_ga_GSD4ETCY1D':'GS1.1.1686902651.1.1.1686903252.28.0.0'
# }


headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
    'Accept':'application/json, text/plain, */*',
    'Accept-Language': 'en-US,en;q=0.9,vi;q=0.8,th;q=0.7,zh-CN;q=0.6,zh;q=0.5',
    'X-Guest-Token': 'Ae1PtIuUDbozkgpQ6VOW57jKqvchZxF2',
    'Connection' : 'keep-alive',
    'TE': 'Trailers'
}

In [8]:
# https://tiki.vn/phu-kien-nam-tui-xach/c958
# https://tiki.vn/vi-nam-ngang/c49652 

In [9]:
list_url = ['https://tiki.vn/tui-xach-cong-so-nam/c5337',
            'https://tiki.vn/tui-bao-tu-tui-deo-bung/c49650',
            'https://tiki.vn/vi-nam-dung/c49654'
            ]

list_page_50 = []
for url in list_url:
    parsed_url = urlparse(url)
    path_segments = parsed_url.path.split('/')

    category_name = path_segments[1]
    category_id = path_segments[2][1:]

    params = {
        'limit':'40',  #uptosize
        'include': 'advertisement',
        'aggregations': '2',
        'trackity_id': '74634137-8702-8b12-fe42-46a46a2a2573',
        'category': category_id,  #change
        'page': '1',
        'urlKey': category_name #change
    }

    product_data = []
    page_error = []
    response = requests.get('https://tiki.vn/api/personalish/v1/blocks/listings', headers=headers, params=params)
    last_page = response.json().get('paging').get('last_page')
    print('total pages',last_page)
    print('CATEGORY ', category_name)
    if last_page == 50:
        list_page_50.append(category_name)
        continue
    else:
        for i in range(1, last_page + 1):
            params['page'] = i
            response = requests.get('https://tiki.vn/api/personalish/v1/blocks/listings', headers=headers, params=params)

            if response.status_code == 200:
                print('request done page', i)
                for record in response.json().get('data'):
                    product_id = record.get('id')
                    product_data.append({'product_id': product_id, 'category_name': category_name, 'max_page': last_page})
            else:
                print('cannot request page',i)
                page_error.append(i)
            time.sleep(random.randrange(3, 10))

        product_df = pd.DataFrame(product_data,columns = ['product_id','category_name','max_page'])  
    
        output_file_name = f'TUI-NAM-{category_name}.csv'
        product_df.to_csv(output_file_name)

page_50 = 'page50.txt'
cannot_request_page = 'page_error.txt'
# Open the file in write mode
with open(page_50, 'a') as file:
    # Write each item of the list to a separate line in the file
    file.writelines(category_name)
    file.write('\n')
    file.writelines('\n'.join(list_page_50))
    file.write('\n')
with open(cannot_request_page, 'a') as file:
    # Write each item of the list to a separate line in the file
    file.writelines('\n'.join(page_error))
    file.write('\n')

total pages 34
CATEGORY  tui-xach-cong-so-nam
request done page 1
request done page 2
request done page 3
request done page 4
request done page 5
request done page 6
request done page 7
request done page 8
request done page 9
request done page 10
request done page 11
request done page 12
request done page 13
request done page 14
request done page 15
request done page 16
request done page 17
request done page 18
request done page 19
request done page 20
request done page 21
request done page 22
request done page 23
request done page 24
request done page 25
request done page 26
request done page 27
request done page 28
request done page 29
request done page 30
request done page 31
request done page 32
request done page 33
request done page 34
total pages 9
CATEGORY  tui-bao-tu-tui-deo-bung
request done page 1
request done page 2
request done page 3
request done page 4
request done page 5
request done page 6
request done page 7
request done page 8
request done page 9
total pages 48
CATEGO

AttributeError: '_io.TextIOWrapper' object has no attribute 'writeline'

In [2]:
page_error

NameError: name 'page_error' is not defined